In [28]:
import torch
from segment_anything.modeling.mask_decoder import MaskDecoder
from segment_anything.build_sam import _build_sam
torch.cuda.is_available()

True

In [29]:
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device:{device}")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.backends.cudnn.enabled = True

using device:cuda


In [32]:
from Hu.Gleason import *
import Hu.utils as utils
dataset =  get_dataset(imgdir='../../data/TrainImgs',
                          maskdir='../../data/labels',
                          train=True,
                          val=False,
                          test=False,
                          transforms=get_transform(train=True))


In [33]:
dataset[0]

(tensor([[[ 2.2489,  2.2489,  2.2489,  ..., -2.1179, -2.1179, -2.1179],
          [ 2.2489,  2.2489,  2.2489,  ..., -2.1179, -2.1179, -2.1179],
          [ 2.2489,  2.2489,  2.2489,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[ 2.4286,  2.4286,  2.4286,  ..., -2.0357, -2.0357, -2.0357],
          [ 2.4286,  2.4286,  2.4286,  ..., -2.0357, -2.0357, -2.0357],
          [ 2.4286,  2.4286,  2.4286,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[ 2.6400,  2.6400,  2.6400,  ..., -1.8044, -1.8044, -1.8044],
          [ 2.6400,  2.6400,

In [19]:
train_sampler = torch.utils.data.RandomSampler(dataset)
data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2,
        sampler=train_sampler, num_workers=10,
        collate_fn=utils.collate_fn, drop_last=True)

In [20]:
for step, (image, gt, bboxes, names_temp) in enumerate(data_loader):
    print(image.shape, gt.shape, bboxes)
    # show the example
    _, axs = plt.subplots(1, 2, figsize=(25, 25))
    idx = random.randint(0, 1)
    axs[0].imshow(image[idx].cpu().permute(1, 2, 0).numpy())
    gt = F.interpolate(gt.float(), size=(1024, 1024), mode="nearest").long()  # shape: (2, 1, 1024, 1024)
    bboxes = bboxes*(1024/256)
    show_mask(gt[idx].cpu().numpy(), axs[0])
    show_box(bboxes[idx].numpy(), axs[0])
    axs[0].axis("off")
    # set title
    axs[0].set_title(names_temp[idx])
    idx = random.randint(0, 1)
    axs[1].imshow(image[idx].cpu().permute(1, 2, 0).numpy())
    show_mask(gt[idx].cpu().numpy(), axs[1])
    show_box(bboxes[idx].numpy(), axs[1])
    axs[1].axis("off")
    # set title
    axs[1].set_title(names_temp[idx])
    plt.show()
    plt.subplots_adjust(wspace=0.01, hspace=0)
    plt.close()
    break

ValueError: not enough values to unpack (expected 4, got 2)

In [7]:
checkpoint = '../medsam_vit_b.pth'
pretrained_state_dict = torch.load(checkpoint,map_location=device)

/tmp/ipykernel_192054/2601783359.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(checkpoint,map_location=device)


In [9]:
from segment_anything.modeling import *
from functools import partial
import torch.nn as nn


class MedSAM(nn.Module):
    def __init__(
        self,
        image_encoder,
        mask_decoder,
        prompt_encoder,
    ):
        super().__init__()
        self.image_encoder = image_encoder
        self.mask_decoder = mask_decoder
        self.prompt_encoder = prompt_encoder
        # freeze prompt encoder
        
        for param in self.prompt_encoder.parameters():
            param.requires_grad = False
        
    def forward(self, image, box, dropout_prob=0.2):
        image_embedding = self.image_encoder(image)  # (B, 256, 64, 64)
        image_embedding = F.dropout(image_embedding, p=dropout_prob, training=self.training)

        # do not compute gradients for prompt encoder
        with torch.no_grad():
            box_torch = torch.as_tensor(box, dtype=torch.float32, device=image.device)
            if len(box_torch.shape) == 2:
                box_torch = box_torch[:, None, :]  # (B, 1, 4)
            sparse_embeddings, dense_embeddings = self.prompt_encoder(
                points=None,
                boxes=box_torch,
                masks=None,
            )
        low_res_masks, _ = self.mask_decoder(
            image_embeddings=image_embedding,  # (B, 256, 64, 64)
            image_pe=self.prompt_encoder.get_dense_pe(),  # (1, 256, 64, 64)
            sparse_prompt_embeddings=sparse_embeddings,  # (B, 2, 256)
            dense_prompt_embeddings=dense_embeddings,  # (B, 256, 64, 64)
            multimask_output=True,
        )
        low_res_masks = F.dropout(low_res_masks, p=dropout_prob, training=self.training)
        ori_res_masks = F.interpolate(
            low_res_masks,
            size=(image.shape[2], image.shape[3]),
            mode="bilinear",
            align_corners=False,
        )
        return ori_res_masks

In [10]:
encoder_global_attn_indexes = [2, 5, 8, 11]

num_class_sam_model = MedSAM(
    image_encoder=ImageEncoderViT(
        depth=12,
        embed_dim=768,
        img_size=1024,
        mlp_ratio=4,
        norm_layer=partial(torch.nn.LayerNorm, eps=1e-6),
        num_heads=12,
        patch_size=16,
        qkv_bias=True,
        use_rel_pos=True,
        global_attn_indexes=encoder_global_attn_indexes,
        window_size=14,
        out_chans=256,
    ),
    mask_decoder=MaskDecoder(
        num_multimask_outputs=3,
        transformer=TwoWayTransformer(
            depth=2,
            embedding_dim=256,
            mlp_dim=2048,
            num_heads=8,
        ),
        transformer_dim=256,
        iou_head_depth=3,
        iou_head_hidden_dim=256,
    ),
    prompt_encoder=PromptEncoder(
        embed_dim=256,
        image_embedding_size=(64, 64),
        input_image_size=(1024, 1024),
        mask_in_chans=16,
    )
)

num_class_sam_model = num_class_sam_model.to(device)
num_class_model_state_dict = num_class_sam_model.state_dict()

In [12]:
for name, param in pretrained_state_dict.items():
    if name in num_class_model_state_dict and num_class_model_state_dict[name].shape == param.shape:
        num_class_model_state_dict[name] = param
# 加载适配的state_dict:
num_class_sam_model.load_state_dict(num_class_model_state_dict, strict=False)


<All keys matched successfully>